In [ ]:
from flask import Flask
from flask import request
from flask import jsonify
import numpy as np
import platform
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

app = Flask(__name__)

model = None

def init():
    global model
    model = load_model('svm.model')

@app.route('/')
def hello_world():
    a = platform.python_version()
    return a

@app.route('/upload', methods=['POST'])
def upload_file(): 
    if model ==None:
        init()
    if request.method == 'POST':
        f = request.files['the_file']
        imgPath = 'a.jpg'        
        f.save(imgPath)
        im = image.load_img(imgPath, target_size=(224, 224))
        im = image.img_to_array(im)
        im = np.expand_dims(im, axis=0)
        x = preprocess_input(im)
        preds = model.predict(x)
        result = decode_predictions(preds, top=3)[0]
        resultdict = {}
        if len(result) > 0:
            resultdict[result[0][1]] = float(result[0][2])
        if len(result) > 1:
            resultdict[result[1][1]] = float(result[1][2])
        if len(result) > 2:
            resultdict[result[2][1]] = float(result[2][2])
        else:
            resultdict['null'] = '0'
        return jsonify(resultdict)

if __name__ == '__main__':
    app.run(host="0.0.0.0",port=5000)